https://adventofcode.com/2020/day/16

In [1]:
import re

In [2]:
datafile = 'data/16-1.txt'

In [3]:
with open(datafile) as fh:
    data_txt = fh.read()

In [4]:
fd_txt, yt_txt, nt_txt = data_txt.split('\n\n')

In [5]:
legit_ranges = []
for line in fd_txt.strip().split('\n'):
    a1, z1, a2, z2 = (int(x) for x in re.findall(r'\d+', line))
    legit_ranges.extend([range(a1, z1+1), range(a2, z2+1)])

In [6]:
nearby_tickets = []
for line in nt_txt.strip().split('\n')[1:]:
    nearby_tickets.append([int(x) for x in line.split(',')])

In [7]:
yt_txt

'your ticket:\n137,173,167,139,73,67,61,179,103,113,163,71,97,101,109,59,131,127,107,53'

In [8]:
my_ticket = [int(x) for x in yt_txt.split('\n')[1].split(',')]
my_ticket

[137,
 173,
 167,
 139,
 73,
 67,
 61,
 179,
 103,
 113,
 163,
 71,
 97,
 101,
 109,
 59,
 131,
 127,
 107,
 53]

In [9]:
%%time
error_rate = 0
for nt in nearby_tickets:
    for v in nt:
        for r in legit_ranges:
            if v in r:
                break
        else:
            error_rate += v

CPU times: user 2.62 ms, sys: 0 ns, total: 2.62 ms
Wall time: 2.63 ms


In [10]:
error_rate

26026

In [11]:
def is_valid(ticket, legit_ranges=legit_ranges):
    for x in ticket:
        for r in legit_ranges:
            if x in r:
                break
        else:
            return False
    return True

In [12]:
is_valid(my_ticket)

True

In [13]:
valid_tickets = [x for x in nearby_tickets if is_valid(x)]
valid_tickets.append(my_ticket)

In [14]:
len(nearby_tickets), len(valid_tickets)

(245, 191)

In [15]:
fields = []
for line in fd_txt.strip().split('\n'):
    fieldname = line.split(':')[0]
    a1, z1, a2, z2 = (int(x) for x in re.findall(r'\d+', line))
    fields.append((fieldname, range(a1, z1+1), range(a2, z2+1)))

In [16]:
len(fields), fields[:3]

(20,
 [('departure location', range(33, 431), range(456, 968)),
  ('departure station', range(42, 865), range(875, 958)),
  ('departure platform', range(42, 806), range(821, 969))])

In [17]:
def is_in_field(v, field):
    return v in field[1] or v in field[2]

In [18]:
fields[0]

('departure location', range(33, 431), range(456, 968))

In [19]:
is_in_field(457, fields[0])

True

In [45]:
fieldsets = [{i for i in range(len(fields))} for _ in fields]
for t in valid_tickets:
    for i, v in enumerate(t):
        for j, field in enumerate(fields):
            if not is_in_field(v, field):
                fieldsets[j].discard(i)

visited = set()
while any(len(x) > 1 for x in fieldsets):
    single = next(y for y in (next(iter(x)) for x in fieldsets if len(x) == 1) if y not in visited)
    visited.add(single)
    for fieldset in fieldsets:
        if len(fieldset) > 1:
            fieldset.discard(single)

In [46]:
field_indices = [x.pop() for x in fieldsets]

In [55]:
field_indices

[1, 3, 10, 6, 8, 19, 5, 16, 12, 2, 11, 18, 4, 7, 13, 14, 9, 17, 15, 0]

In [64]:
properly_sorted_fields = [x[1] for x in sorted(zip(field_indices, fields))]

In [56]:
# properly_sorted_fields = [None] * len(field_indices)

In [57]:
# for i, j in enumerate(field_indices):
#     properly_sorted_fields[j] = fields[i]

In [47]:
# properly_sorted_fields = [fields[i] for i in field_indices]

In [65]:
properly_sorted_fields

[('zone', range(34, 189), range(212, 960)),
 ('departure location', range(33, 431), range(456, 968)),
 ('arrival track', range(28, 341), range(349, 969)),
 ('departure station', range(42, 865), range(875, 958)),
 ('price', range(48, 923), range(939, 952)),
 ('arrival location', range(50, 488), range(507, 955)),
 ('departure track', range(34, 75), range(93, 968)),
 ('route', range(33, 643), range(666, 961)),
 ('departure date', range(40, 400), range(417, 956)),
 ('train', range(50, 605), range(630, 972)),
 ('departure platform', range(42, 806), range(821, 969)),
 ('class', range(49, 525), range(543, 952)),
 ('arrival platform', range(42, 730), range(751, 960)),
 ('row', range(39, 239), range(255, 974)),
 ('seat', range(48, 149), range(161, 974)),
 ('wagon', range(45, 899), range(921, 967)),
 ('arrival station', range(34, 694), range(718, 957)),
 ('type', range(29, 300), range(316, 953)),
 ('duration', range(40, 373), range(397, 952)),
 ('departure time', range(30, 775), range(797, 951))

In [66]:
for t in valid_tickets:
    for v, f in zip(t, properly_sorted_fields):
        if not is_in_field(v, f):
            print(v, f)    

In [68]:
my_ticket_fields = list(zip((x[0] for x in properly_sorted_fields), my_ticket))
my_ticket_fields

[('zone', 137),
 ('departure location', 173),
 ('arrival track', 167),
 ('departure station', 139),
 ('price', 73),
 ('arrival location', 67),
 ('departure track', 61),
 ('route', 179),
 ('departure date', 103),
 ('train', 113),
 ('departure platform', 163),
 ('class', 71),
 ('arrival platform', 97),
 ('row', 101),
 ('seat', 109),
 ('wagon', 59),
 ('arrival station', 131),
 ('type', 127),
 ('duration', 107),
 ('departure time', 53)]

In [61]:
departure_fields = [y for (x, y) in my_ticket_fields if x.startswith('departure')]
departure_fields

[173, 139, 61, 103, 163, 53]

In [70]:
part_2 = 1
for name, val in my_ticket_fields:
    if name.startswith('departure'):
        part_2 *= val
part_2

1305243193339

## post

In [114]:
%%time

import re

with open(datafile) as fh:
    data_txt = fh.read()
fd_txt, yt_txt, nt_txt = data_txt.split('\n\n')

my_ticket = [int(x) for x in yt_txt.split('\n')[1].split(',')]

nearby_tickets = []
for line in nt_txt.strip().split('\n')[1:]:
    nearby_tickets.append([int(x) for x in line.split(',')])

fields = []
for line in fd_txt.strip().split('\n'):
    fieldname = line.split(':')[0]
    a1, z1, a2, z2 = (int(x) for x in re.findall(r'\d+', line))
    fields.append((fieldname, a1, z1, a2, z2))


def is_in_field(v, f):
    return f[1] <= v <= f[2] or f[3] <= v <= f[4]


error_rate = 0
for nt in nearby_tickets:
    for v in nt:
        if not any(is_in_field(v, f) for f in fields):
            error_rate += v
            
part_1 = error_rate


def is_valid(ticket, fields=fields):
    for v in ticket:
        if not any(is_in_field(v, f) for f in fields):
            return False
    return True


valid_tickets = [x for x in nearby_tickets if is_valid(x)]
valid_tickets.append(my_ticket)

nfields = len(fields)
fieldsets = [set(range(nfields)) for _ in range(nfields)]
for t in valid_tickets:
    for i, v in enumerate(t):
        for j, field in enumerate(fields):
            if not is_in_field(v, field):
                fieldsets[j].discard(i)

visited = set()
while True:
    try:
        single = next(y for y in (next(iter(x)) for x in fieldsets if len(x) == 1) if y not in visited)
    except StopIteration:
        break
    visited.add(single)
    for fieldset in fieldsets:
        if len(fieldset) > 1:
            fieldset.discard(single)

field_indices = [x.pop() for x in fieldsets]
sorted_fieldnames = [y for (x, y) in sorted(zip(field_indices, (x[0] for x in fields)))]

my_fielded_ticket = list(zip(sorted_fieldnames, my_ticket))

part_2 = 1
for name, v in my_fielded_ticket:
    if name.startswith('departure'):
        part_2 *= v

CPU times: user 26.8 ms, sys: 0 ns, total: 26.8 ms
Wall time: 26 ms


In [113]:
part_1, part_2

(26026, 1305243193339)